## Visualization of constraints for optimizing a contrastive margin loss
(similar for infoNCE, todo later)



### Summary of the constraints (see paper)

linear system $A \log h^T = r$ of three contraints in log-configuration
space, with


\begin{equation}
    \begin{gathered}
        A=
        \begin{pmatrix}
            1 & 0 & -1\\
            0 & 1 & -2\\
            1 & -1 & 0
        \end{pmatrix}
        \ , \\
        \log h^T=
        \begin{pmatrix}
            \log\Lambda_p\\ \log\Lambda_e\\ \log b\\
            \end{pmatrix}
        \, \text{and} \quad
        r=
        \begin{pmatrix}
            \log r_p \\ \log r_e + \log \eta\\ \log r_{eq}
            \end{pmatrix}
    \end{gathered}
    \label{eq:linear_system}
    \end{equation}

We will do a 3D graph with axes: $(x, y, z) = (\log b, \log \Lambda_e, \log \Lambda_p)$. In 'graph axes', 
the constraints become $A \phi = r$, with

\begin{equation}
    \begin{gathered}
        A=
        \begin{pmatrix}
            1 & 0 & -1\\
            0 & 1 & -2\\
            1 & -1 & 0
        \end{pmatrix}
        \ , \\
        \phi=
        \begin{pmatrix}
            z\\ y\\ x\\
            \end{pmatrix}
        \, \text{and} \quad
        r=
        \begin{pmatrix}
            \log r_p \\ \log r_e + \log \eta\\ \log r_{eq}
            \end{pmatrix}
    \end{gathered}
    \label{eq:linear_system}
    \end{equation}
    
(we make this choice for the axis because the 'front plane' is then the same as the 'color plots' graph. 

In [1]:
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
from ipywidgets import widgets, interact, interact_manual, interactive_output

# First, create plot with any data in it

# Constants (arbitrary values) (noting Ri = \log r_i)
Rp = 0
Re = 0
Req = 0
logeta = 0
# axes plotting range. TODO this has to be the same range used in x, y
xyrge = [-2., 3.]
opacity = 0.5
# In the 3D graph we are preparing, x axis will be for \log b, y axis for \log \Lambda_e, 
# and z axis for \log \Lambda_p
# x, y grid to be used for contraints involving z, i.e. first and last line of A, 
# corresponding to C_p and C_eq
x = np.linspace(xyrge[0], xyrge[1], 100)
y = np.linspace(xyrge[0], xyrge[1], 100)
Xgz, Ygz = np.meshgrid(x, y)
# Last constraint does not involve z in its equation ('vertical plane' in z) so we cannot plot
# if in the same grid (x, y), 
# let's use an x, z grid for this last constraint, i.e. second line of A, or C_e constraint
z = np.linspace(xyrge[0], xyrge[1], 100)
Xgy, Zgy = np.meshgrid(x, z)
# Constraints
# on Z
ZP = Xgz + Rp  # (C_p)
ZEQ = Ygz + Req  # (C_eq)
# on Y
YE = 2 * Xgy + Re + logeta  # (C_e)
# This time let's do just one plot with everything on it
d = {'is_3d': True}
specs = [[d]]
# Plotting
fig = go.Figure()
sP = go.Surface(x=Xgz, y=Ygz, z=ZP, opacity=opacity, colorscale='reds')
sEQ = go.Surface(x=Xgz, y=Ygz, z=ZEQ, opacity=opacity, colorscale='greens')
sE = go.Surface(x=Xgy, y=YE, z=Zgy, opacity=opacity, colorscale='blues')
# add surface
fig.add_trace(sP)
fig.add_trace(sEQ)
fig.add_trace(sE)
# Let's also plot intersection lines between each pair of constraints
# C_p and Ceq
xlineP_EQ = x; ylineP_EQ = xlineP_EQ + Rp - Req; zlineP_EQ = xlineP_EQ + Rp; 
fig.add_trace(go.Scatter3d(x=xlineP_EQ, y=ylineP_EQ, z=zlineP_EQ, mode='lines', line={'color': 'blue', 'width': 3}))
# C_p and C_e
xlineP_E = x; ylineP_E = 2 * xlineP_E + Re + logeta ; zlineC_E = xlineP_E + Rp;
fig.add_trace(go.Scatter3d(x=xlineP_E, y=ylineP_E, z=zlineC_E, mode='lines', line={'color': 'green', 'width': 3}))
# C_eq and C_e
xlineEQ_E = x; ylineEQ_E = 2 * xlineEQ_E + Re + logeta; zlineEQ_E = ylineEQ_E + Req
fig.add_trace(go.Scatter3d(x=xlineEQ_E, y=ylineEQ_E, z=zlineEQ_E, mode='lines', line={'color': 'red', 'width': 3}))
# Title (disabled now, better in Latex)
def get_title(Req, Re, Rp, logeta):
    return f"Representation of 3 constraints in 3D configuration space. Here log req={Req}, Re={Re}, Rp={Rp}, log eta={logeta} ." 
# fig.update_layout(title=get_title(Req, Re, Rp, logeta))  
# axes labels
# It is a known limitation that 3d plots in plotly do not support Latex labels: 
# But what works it to simply copy paste (from markdown rendering)
d_axtitle = dict(xaxis_title="log 𝑏", yaxis_title='log Λ𝑒', zaxis_title='log Λ𝑝')
fig.update_layout(**{f'scene{i}': d_axtitle for i in range(1, 6)})
# scene aspect
fig.update_layout(**{f'scene_aspectmode': 'cube'}) # 'data' or 'cube'
# Scaling and axis ranges
fig.update_layout(**{f'scene.{a}axis.range': xyrge for a in ['x', 'y', 'z']})
fig.update_layout(height=700)
# remove colorbar and line legend
fig.update_traces(selector='surface', showscale=False)
fig.update_layout(showlegend=False)
# And I think now we don't need an interactive plot anymore
# Remove axis ticklabels, ranges are arbitrary anyways
d_upd_ticklabels = {f'scene_{w}axis_showticklabels': False for w in ['x', 'y', 'z']}
fig.update_layout(**d_upd_ticklabels)
# Define 'orientation' of view
zoom = 2.2
camera = dict(eye=dict(x=4./zoom, y=3./zoom, z=1.7/zoom))
fig.update_layout(scene_camera=camera)
# remove white margins around graph
fig.update_layout(margin_l=0, margin_r=0, margin_t=0, margin_b=0)
# Save to different formats (take care about margins ?)
fig.update_layout(font_size=20)
fig.write_image("constraints.pdf")
fig.write_html("constraints.html")
